In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [12]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)
path_to_file = os.path.join(path_to_zip)+"/spa-eng/spa.txt"

In [6]:
path_to_file

'/root/.keras/datasets/spa-eng_extracted/spa-eng/spa.txt'

In [145]:
# with open(path_to_file,encoding='utf-8') as f:
#     df=f.read().strip().split('\n')

In [146]:
df[:10]

['Hi.\tمرحبًا.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #629296 (Samer)',
 'Run!\tاركض!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #1245450 (saeb)',
 'Duck!\tاخفض رأسك!\tCC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036391 (KeEichi)',
 'Duck!\tاخفضي رأسك!\tCC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036392 (KeEichi)',
 'Duck!\tاخفضوا رؤوسكم!\tCC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036393 (KeEichi)',
 'Help!\tالنجدة!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #371293 (saeb)',
 'Jump!\tاقفز!\tCC-BY 2.0 (France) Attribution: tatoeba.org #1102981 (jamessilver) & #6009426 (damascene)',
 'Stop!\tقف!\tCC-BY 2.0 (France) Attribution: tatoeba.org #448320 (CM) & #1245447 (saeb)',
 'Stop!\tتوقف !\tCC-BY 2.0 (France) Attribution: tatoeba.org #448320 (CM) & #5496702 (Wildflower81)',
 'Wait!\tإنتظر\tCC-BY 2.0 (France) Attribution: tatoeba.org #1744314 (belgavox) & #5496709 (Wildflow

In [147]:
df[0].split('\t')

['Hi.',
 'مرحبًا.',
 'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #629296 (Samer)']

In [148]:
len(df)

12569

In [149]:
import re

In [150]:
re.sub(r"([?.!,¿])", r" \1 ", "Go.")

'Go . '

In [151]:
def preprocess_sentence(w):
    w = re.sub(r"([?.!،,¿])", r" \1 ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    # w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.lower().strip()
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [152]:
preprocess_sentence("hello from the other sidE.")

'<start> hello from the other side . <end>'

In [230]:
def create_dataset(path_to_file,num_examples=None):
    with open(path_to_file,encoding='utf-8') as f:
        df=f.read().strip().split('\n')
    pairs=[[preprocess_sentence(sent) for sent in line.split('\t')]for line in df[:num_examples]]
    return zip(*pairs)

In [232]:
eng,ara,_=create_dataset(path_to_file)

In [233]:
len(eng)

12569

In [234]:
eng[-1]

"<start> there are mothers and fathers who will lie awake after the children fall asleep and wonder how they'll make the mortgage ,  or pay their doctor's bills ,  or save enough for their child's college education . <end>"

In [235]:
spa[-1]

'<start> وهناك أمهات وآباء سيظلون مستيقظين بعد أن ينام أطفالهم ،  يتساءلون عن كيف سيسددون أقساط الرهن العقاري الذي اشترَوْ به بيتهم ،  وكيف سيدفعون فواتير أطبائهم ،  أو توفير ما يحتاجونه من مال لتسديد رسوم تسجيل أبنائهم في الجامعات . <end>'

In [251]:
def custom_standardize(text):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, r"[^\w<> ]", "")  # keep < and >
    return text
    
src_vectorizer = tf.keras.layers.TextVectorization(
    standardize=custom_standardize,

    output_mode="int",
    # output_sequence_length=30
)
src_vectorizer.adapt(eng)
input_tensor = src_vectorizer(eng)

In [252]:
def arabic_standardize(text):
    text = tf.strings.strip(text)                  # remove leading/trailing spaces
    text = tf.strings.lower(text)                  # lowercase (works for Arabic letters)
    # text = tf.strings.regex_replace(text, r"[^\u0600-\u06FF\s]", "")  # keep Arabic letters and spaces
    text = tf.strings.regex_replace(text, r"\s+", " ")                 # collapse multiple spaces
    return text

In [ ]:
tgt_vectorizer = tf.keras.layers.TextVectorization(
    standardize=arabic_standardize,
    output_mode="int",
        # output_sequence_length=30

)
tgt_vectorizer.adapt(ara)
target_tensor = tgt_vectorizer(ara)

decoder_inp = target_tensor[:, :-1]
decoder_tar = target_tensor[:, 1:]

eng_vocab = len(src_vectorizer.get_vocabulary())
ara_vocab = len(tgt_vectorizer.get_vocabulary())

In [257]:
eng_vocab

4418

In [ ]:
ara_vocab

13512

# Tensorflow pipline

In [259]:
buffer_size=len(input_tensor) 
batch_size=64

In [260]:
dataset = (
    tf.data.Dataset.from_tensor_slices(((input_tensor, decoder_inp), decoder_tar))
    .shuffle(buffer_size)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# Model Building & Training

In [261]:
from tensorflow.keras import Model,Input
from tensorflow.keras import layers

In [272]:
# Encoder
units=64
embed_dim=100
# Encoder
encoder_inputs=Input(shape=(None,))
x=layers.Embedding(input_dim=eng_vocab,output_dim=embed_dim,name='Encoder_Embed')(encoder_inputs)
encoder_outputs ,state_h,state_c= layers.LSTM(units,return_state=True,name='Encoder_LSTM')(x)
encoder_states=[state_h,state_c]

In [ ]:
# Decoder
decoder_inputs=Input(shape=(None,))
x=layers.Embedding(input_dim=ara_vocab,output_dim=embed_dim,name='Decoder_Embed')(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True,name='Decoder_LSTM')
decoder_outputs, _, _ = decoder_lstm(x, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(ara_vocab, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [307]:
# Seq2Seq Model
model=Model(inputs=[encoder_inputs,decoder_inputs], outputs=decoder_outputs)

In [345]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [299]:
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_11      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Encoder_Embed       │ (None, None, 100) │    441,800 │ input_layer_10[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder_Embed       │ (None, None, 100) │  1,351,200 │ input_layer_11[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Encoder_LSTM (LSTM) │ [(None, 64),      │     42,240 │ Encoder_Embed[0]… │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder_LSTM (LSTM) │ [(None, None,     │     42,240 │ Decoder_Embed[0]… │
│                     │ 64), (None, 64),  │            │ Encoder_LSTM[0][… │
│                     │ (None, 64)]       │            │ Encoder_LSTM[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, None,      │    878,280 │ Decoder_LSTM[0][… │
│                     │ 13512)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,755,760 (10.51 MB)

 Trainable params: 2,755,760 (10.51 MB)

 Non-trainable params: 0 (0.00 B)

In [305]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    filepath='seq2seq_epoch.h5',
    # filepath='seq2seq_epoch_{epoch:02d}.h5',
    save_weights_only=False,
    save_freq='epoch'
)

In [344]:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/working/seq2seq_epoch.h5')

In [346]:
model.fit(dataset, epochs=30, verbose=1,callbacks=[checkpoint])

Epoch 1/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9708 - loss: 0.1387

197/197 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.9708 - loss: 0.1387
Epoch 2/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9717 - loss: 0.1354

197/197 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9717 - loss: 0.1354
Epoch 3/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9723 - loss: 0.1324

197/197 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9723 - loss: 0.1324
Epoch 4/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9728 - loss: 0.1312

197/197 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9728 - loss: 0.1312
Epoch 5/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9733 - loss: 0.1283

...

197/197 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9816 - loss: 0.0896
Epoch 27/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9819 - loss: 0.0891

197/197 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9819 - loss: 0.0891
Epoch 28/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9815 - loss: 0.0888

197/197 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9815 - loss: 0.0888
Epoch 29/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9821 - loss: 0.0864

197/197 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9821 - loss: 0.0864
Epoch 30/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9824 - loss: 0.0840

197/197 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9824 - loss: 0.0841


In [115]:
# model.fit(
#     x=[input_tensor,decoder_inp],y= tf.expand_dims(decoder_tar, -1),
#     batch_size=128,
#     epochs=30,
#     verbose=1
# )

# Inference model


In [347]:
# Encoder model
encoder_model=Model(encoder_inputs,encoder_states)

In [348]:
# decoder model
decoder_state_input_h = tf.keras.Input(shape=(units,))
decoder_state_input_c = tf.keras.Input(shape=(units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [349]:
# send to decoder embedding layer
dec_embed = model.layers[3](decoder_inputs)  #<start>
dec_output,dec_h,dec_c=decoder_lstm(dec_embed,initial_state=decoder_states_inputs) #<states of sentence>
dec_output=decoder_dense(dec_output)
dec_states=[dec_h,dec_c]
decoder_model=Model(inputs=[decoder_inputs] + decoder_states_inputs,outputs= [dec_output]+ dec_states)

In [350]:
decoder_model.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder_Embed       │ (None, None, 100) │  1,351,200 │ input_layer_11[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_18      │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_19      │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder_LSTM (LSTM) │ [(None, None,     │     42,240 │ Decoder_Embed[1]… │
│                     │ 64), (None, 64),  │            │ input_layer_18[0… │
│                     │ (None, 64)]       │            │ input_layer_19[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, None,      │    878,280 │ Decoder_LSTM[3][… │
│                     │ 13512)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,271,720 (8.67 MB)

 Trainable params: 2,271,720 (8.67 MB)

 Non-trainable params: 0 (0.00 B)

In [351]:
# reverse_tgt_vocab = tgt_vectorizer.get_vocabulary()

In [352]:
# idx_to_word = dict(enumerate(reverse_tgt_vocab))
# word_to_idx = {w:i for i,w in idx_to_word.items()}

In [369]:
def decode_sequence(input_sentence,max_len=10):
    states_value=encoder_model.predict(src_vectorizer([input_sentence]))
    # 
    vocab = tgt_vectorizer.get_vocabulary()
    start_idx = vocab.index("<start>")
    end_idx = vocab.index("<end>")

    target_seq = np.array([[start_idx]], dtype="int32")
    decoded_sentence = []
    for _ in range(max_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = vocab[sampled_token_index]
        if sampled_word == "<end>":
            break

        decoded_sentence.append(sampled_word)
        target_seq = np.array([[sampled_token_index]], dtype="int32")
        states_value = [h, c]

    return " ".join(decoded_sentence)

In [370]:
print(decode_sequence("How are you"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
هل هذه القدم هذه هي يكن لي هي يبقَ أقساط


In [371]:
print(decode_sequence("please"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
من فضلك


In [378]:
print(decode_sequence("My name is "))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
هو هي يبقَ كبيرا عن توفير
